<a href="https://colab.research.google.com/github/VishnuPy/MyPython/blob/master/RecommendationApplication_Using_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
%ls

Found GPU at: /device:GPU:0
Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive
'My Drive'/


In [54]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.5381055770003513
GPU (s):
0.19531461200040212
GPU speedup over CPU: 18x


In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv("/gdrive/My Drive/MyLearning/MLDLAIPython/Data/TextData/movie_metadata.csv")

In [57]:
data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,8,143,NaN,0.0,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


<font color=brown size=4>
Check on null values

In [58]:
data.isnull().sum(axis=0)

color                         19
director_name                104
num_critic_for_reviews        50
duration                      15
director_facebook_likes      104
actor_3_facebook_likes        23
actor_2_name                  13
actor_1_facebook_likes         7
gross                        884
genres                         0
actor_1_name                   7
movie_title                    0
num_voted_users                0
cast_total_facebook_likes      0
actor_3_name                  23
facenumber_in_poster          13
plot_keywords                153
movie_imdb_link                0
num_user_for_reviews          21
language                      12
country                        5
content_rating               303
budget                       492
title_year                   108
actor_2_facebook_likes        13
imdb_score                     0
aspect_ratio                 329
movie_facebook_likes           0
dtype: int64

In [59]:
print ('Actor 1 missing values ', data["actor_1_name"].isnull().sum())
print ('Actor 2 missing values ', data["actor_2_name"].isnull().sum())
print ('Actor 3 missing values ', data["actor_3_name"].isnull().sum())
print ('Director missing values ', data["director_name"].isnull().sum())

Actor 1 missing values  7
Actor 2 missing values  13
Actor 3 missing values  23
Director missing values  104


In [0]:
#fill the missing values of the significant features like Actions, Directors 

data["actor_1_name"] = data["actor_1_name"].replace(np.nan, 'unknown')
data["actor_2_name"] = data["actor_2_name"].replace(np.nan, 'unknown')
data["actor_3_name"] = data["actor_3_name"].replace(np.nan, 'unknown')
data["director_name"] = data["director_name"].replace(np.nan, 'unknown')

In [61]:
print(data['genres'], data['genres'].str.find("|"))

0       Action|Adventure|Fantasy|Sci-Fi
1              Action|Adventure|Fantasy
2             Action|Adventure|Thriller
3                       Action|Thriller
4                           Documentary
                     ...               
5038                       Comedy|Drama
5039       Crime|Drama|Mystery|Thriller
5040              Drama|Horror|Thriller
5041               Comedy|Drama|Romance
5042                        Documentary
Name: genres, Length: 5043, dtype: object 0       6
1       6
2       6
3       6
4      -1
       ..
5038    6
5039    5
5040    5
5041    6
5042   -1
Name: genres, Length: 5043, dtype: int64


In [0]:
#Genre has a seperator "pipe" "|", this should be replaced with a white space to make them separate words
data['genres'] = data['genres'].str.replace("|", " ")

In [63]:
data['genres'].head()

0    Action Adventure Fantasy Sci-Fi
1           Action Adventure Fantasy
2          Action Adventure Thriller
3                    Action Thriller
4                        Documentary
Name: genres, dtype: object

In [64]:
data['movie_title']

0                                                 Avatar 
1               Pirates of the Caribbean: At World's End 
2                                                Spectre 
3                                  The Dark Knight Rises 
4       Star Wars: Episode VII - The Force Awakens    ...
                              ...                        
5038                             Signed Sealed Delivered 
5039                           The Following             
5040                                A Plague So Pleasant 
5041                                    Shanghai Calling 
5042                                   My Date with Drew 
Name: movie_title, Length: 5043, dtype: object

In [0]:
#convert Movie Title to Lower case for matching searches
data['movie_title'] = data['movie_title'].str.lower()

In [66]:
data['movie_title'].head()

0                                              avatar 
1            pirates of the caribbean: at world's end 
2                                             spectre 
3                               the dark knight rises 
4    star wars: episode vii - the force awakens    ...
Name: movie_title, dtype: object

In [67]:
data['movie_title'].str[:-1]

0                                                  avatar
1                pirates of the caribbean: at world's end
2                                                 spectre
3                                   the dark knight rises
4       star wars: episode vii - the force awakens    ...
                              ...                        
5038                              signed sealed delivered
5039                            the following            
5040                                 a plague so pleasant
5041                                     shanghai calling
5042                                    my date with drew
Name: movie_title, Length: 5043, dtype: object

In [68]:
print (data['movie_title'][0], len(data['movie_title'][0]))
print (data['movie_title'][2], len(data['movie_title'][2]))
print (data['movie_title'][4], len(data['movie_title'][4]))
print (data['movie_title'][10], len(data['movie_title'][10]))
print (data['movie_title'][5000], len(data['movie_title'][5000]))

avatar  7
spectre  8
star wars: episode vii - the force awakens              55
batman v superman: dawn of justice  35
raymond did it  15


In [0]:
# All movie titles have one additional character. Hence remove last character from the Movie Title
data['movie_title'] = data['movie_title'].str[:-1]

In [81]:
print (data['movie_title'][0], len(data['movie_title'][0]))
print (data['movie_title'][2], len(data['movie_title'][2]))
print (data['movie_title'][4], len(data['movie_title'][4]))
print (data['movie_title'][10], len(data['movie_title'][10]))
print (data['movie_title'][5000], len(data['movie_title'][5000]))

avatar 6
spectre 7
star wars: episode vii - the force awakens             54
batman v superman: dawn of justice 34
raymond did it 14


<font color=brown size=4>
Recommender Applications work on the Similarity concept, that is, depends on the users historical choices (Content Based Systems) and/or the most voted (Popularity Based Systems) and/or similar profile users most watched (Collaborative Filtering Based Systems). Content Based Recommender Systemswork on the Similarity between two different words/phases. Mathmatically, Cosine is what needs to be calculated for measuring the similarity. Four step approach 1) Count the number of Unique Words (Vector 1) 2) Count the Frequency of each word in the text (Vector 2). 3) Plot these two on the graph with words as X axis and Frequency as Measure. 4) Calculate Cosine Angle between these two Vectors. Similarity is between 0 and 1 and more closer to 1, the more similar both vectors are. 

In [71]:
data['actor_2_name']+' '+data['actor_1_name']+' '+data['actor_3_name']+' '+data['genres']+' '+data['director_name']

0       Joel David Moore CCH Pounder Wes Studi Action ...
1       Orlando Bloom Johnny Depp Jack Davenport Actio...
2       Rory Kinnear Christoph Waltz Stephanie Sigman ...
3       Christian Bale Tom Hardy Joseph Gordon-Levitt ...
4       Rob Walker Doug Walker unknown Documentary Dou...
                              ...                        
5038    Daphne Zuniga Eric Mabius Crystal Lowe Comedy ...
5039    Valorie Curry Natalie Zea Sam Underwood Crime ...
5040    Maxwell Moody Eva Boehnke David Chandler Drama...
5041    Daniel Henney Alan Ruck Eliza Coupe Comedy Dra...
5042    Brian Herzlinger John August Jon Gunn Document...
Length: 5043, dtype: object

In [0]:
#To enable Similarity Measurement, all Significant Text Fields are to be concatenated

data['comb'] = data['actor_2_name']+' '+data['actor_1_name']+' '+data['actor_3_name']+' '+data['genres']+' '+data['director_name']

In [73]:
data['comb'].head()

0    Joel David Moore CCH Pounder Wes Studi Action ...
1    Orlando Bloom Johnny Depp Jack Davenport Actio...
2    Rory Kinnear Christoph Waltz Stephanie Sigman ...
3    Christian Bale Tom Hardy Joseph Gordon-Levitt ...
4    Rob Walker Doug Walker unknown Documentary Dou...
Name: comb, dtype: object

<font color=brown size=4>
Now will have to calculate Similarity, Count Matrix is to be created first and then Similarity Matrix. 
#Similarity Matrix contains the cosine similarity of all the movies in the dataset. We can get the score by referring it by Index in the dataset

In [0]:
#importing libraries for making count matrix and similarity matrix

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
#Count Matrix
cv = CountVectorizer()
count_matrix = cv.fit_transform(data['comb'])

#Similarity Matrix
sim = cosine_similarity(count_matrix)

In [79]:
list(enumerate(sim[0]))

[(0, 0.9999999999999999),
 (1, 0.24174688920761409),
 (2, 0.16116459280507606),
 (3, 0.08058229640253803),
 (4, 0.0),
 (5, 0.3086066999241839),
 (6, 0.253546276418555),
 (7, 0.14285714285714288),
 (8, 0.2964997266644405),
 (9, 0.2314550249431379),
 (10, 0.3086066999241839),
 (11, 0.3086066999241839),
 (12, 0.16903085094570333),
 (13, 0.24174688920761409),
 (14, 0.16116459280507606),
 (15, 0.37062465833055064),
 (16, 0.2314550249431379),
 (17, 0.2964997266644405),
 (18, 0.24174688920761409),
 (19, 0.3450327796711771),
 (20, 0.253546276418555),
 (21, 0.24174688920761409),
 (22, 0.14285714285714288),
 (23, 0.253546276418555),
 (24, 0.15430334996209194),
 (25, 0.15430334996209194),
 (26, 0.16903085094570333),
 (27, 0.2964997266644405),
 (28, 0.2964997266644405),
 (29, 0.28571428571428575),
 (30, 0.16116459280507606),
 (31, 0.3223291856101521),
 (32, 0.2964997266644405),
 (33, 0.16116459280507606),
 (34, 0.3571428571428572),
 (35, 0.14824986333222026),
 (36, 0.3086066999241839),
 (37, 0.308

<font color=brown size=4>
This application is about Recommending Movie(s) to the user based on the Movie Title entered as input. To simify, we will stick to top 10 Movie Recommendations. Since this is a repeated task, we will make a function that accepts Title as Input and gives out 10 Movie Title as an array

In [0]:
def recommend(m):
    m = m.lower() #coz, we changed all Movie Title in the Dataset to Lower Case
    #First check if the Input movie is in the database
    if m not in data['movie_title'].unique():
        print ("This movie is not in our Database")
    else:
        i = data.loc[data['movie_title']==m].index[0] #get the index of the Movie Title that has been Input by the user
        lst = list(enumerate(sim[i])) #Enumeration is required to retain the Cosine Similarity of the Movie Title
        lst = sorted(lst, key=lambda x:x[1], reverse=True)
        lst = lst[1:11] # started with 1 Index as 0 index is  the movie itself and hence not required
        l = []
        for i in range(len(lst)):
            a = lst[i][0]
            l.append(data['movie_title'][a])
        for i in range (len(l)):
            print (l[i])


In [84]:
recommend("The Avengers")

the avengers
the avengers
captain america: civil war
iron man 2
captain america: the winter soldier
iron man 3
star trek
iron man
serenity
the island
